Mittlere ACF (0–50 Lags)


Langsames Abklingen
Die ACF startet bei 1.0 (Lag 0) und fällt nur langsam ab – schon bei Lag 50 liegt sie noch bei rund 0.75. Das spricht für eine sehr starke Persistenz, d.h. Verbräuche bleiben auch über mehrere Stunden hinweg hoch korreliert.

Bump um Lag 24
Zwischen Lag 20–28 siehst du einen kleinen „Höcker“ (ACF steigt von ~0.78 wieder auf ~0.80). Das ist genau der tägliche Saison-Effekt (24 h), der deine Zeitreihen zusätzlich stützt.

Fehlende deutliche Peaks bei Lag 7×24 oder 168
Da du nur bis Lag 50 geplottet hast, siehst du den Wochen-Peak (168 h) hier noch nicht. Würdest du bis Lag 168 gehen, käme dort ein weiterer, kleiner Bump.

Fazit aus der ACF:

Starkes Grundrauschen mit sehr langsamer Abklingrate

Eindeutige tägliche Saisonkorrelation bei Lag 24

Mittlere PACF (0–50 Lags)


Spitze bei Lag 1
Der PACF-Wert bei Lag 1 ist praktisch 1.0 → deutet auf einen AR(1)-Anteil hin.

Negativer Spike bei Lag 2
Direkt bei Lag 2 geht der PACF kurz unter 0 (rund –0.2) → ein Hinweis darauf, dass auch ein MA(1) – oder kombiniertes ARMA(1,1) – Modell passen könnte.

Kleine Hügel bei Lag 24–25
Im Bereich um Lag 24–25 tauchen im PACF noch einmal kleine positive Ausreißer auf, was auf einen saisonalen AR-Term bei einer Periodenlänge von 24 h hindeutet.

Schnelles Abklingen danach
Ab Lag 30 ist der PACF für die meisten Lags nahe null → hier ist kein weiterführender rein nicht-saisonaler AR-Order mehr nötig.

Fazit aus der PACF:

Ein nicht‐saisonaler AR(1) ist sinnvoll.

Ein MA(1) (bei Lag 2 negativer Effekt) kann helfen, das erste Abklingen zu modellieren.

Ein saisonaler AR(1) bei Lag 24 (für den Tageszyklus) ist gerechtfertigt.

Konsequenz für dein Basis-SARIMAX
Nicht‐saisonaler Teil: wahrscheinliche Kandidaten sind
(

,

,

)
=
(
1
,
0
,
1
)
(p,d,q)=(1,0,1) (da ADF-Test + ACF/PACF stationär ohne Differenzierung).

Saisonaler Teil (m=24): ein

=
1
P=1 aus dem PACF-Hügel bei Lag 24, und ggfs.

=
0
Q=0 (keine klare Cut-Off in der ACF).

Weitere Saisonen (168 h / 8760 h) behandelst du idealerweise exogen (Fourier oder Dummies) oder per MSTL-Features.

Damit hättest du für dein Basis-SARIMAX in etwa:

python
Copy
Edit
seasonal_order=(1,0,0,24)    # P=1, D=0, Q=0, m=24
order=(1,0,1)                # p=1, d=0, q=1
plus deine exogenen Features für Wochen‐ und Jahreseffekte. So schlagen deine ACF/PACF-Erkenntnisse direkt in Parameterempfehlungen um.


In [1]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from utils import load_series_dfs
import pickle

In [2]:
series_dfs = load_series_dfs(data_dir=r"C:\Users\johan\Documents\FH_Master\data", filename="final_learning_dfs.pkl")
series_dfs['FL_00024702.PLAN.MengeHH'].columns

Index(['consumption', 'hour', 'weekday', 'month', 'is_weekend', 'w_tl', 'w_rf',
       'w_ff', 'w_ffx', 'w_cglo', 'w_so_h', 'w_rr', 'w_rrm', 'w_tb10',
       'w_tb20', 'CEGH_WAP', 'THE_WAP'],
      dtype='object')

In [3]:
with open('../data/mstl_results.pkl', 'rb') as f:
    mstl_results = pickle.load(f)

In [11]:
# --- 1) Panel aufbauen wie gehabt, aber ohne series_key ---
rows = []
for key, df in series_dfs.items():
    tmp = df[['consumption',
              'hour','weekday','month','is_weekend',
              'w_tl','w_rf','w_ff','w_ffx','w_cglo','w_so_h',
              'w_rr','w_rrm','w_tb10','w_tb20',
              'CEGH_WAP','THE_WAP']]  # Deine Wetter- und Preis-Features
    m = mstl_results[key].seasonal.rename(columns={
        'seasonal_24':'s24','seasonal_168':'s168','seasonal_8760':'s8760'
    })
    tmp = tmp.join(m)
    tmp['series_key'] = key
    rows.append(tmp)

panel = pd.concat(rows).reset_index()

In [15]:
panel.columns

Index(['von  | von', 'consumption', 'hour', 'weekday', 'month', 'is_weekend',
       'w_tl', 'w_rf', 'w_ff', 'w_ffx', 'w_cglo', 'w_so_h', 'w_rr', 'w_rrm',
       'w_tb10', 'w_tb20', 'CEGH_WAP', 'THE_WAP', 's24', 's168', 's8760',
       'series_key'],
      dtype='object')

In [16]:
panel_exog = panel.drop(columns=['consumption', 'von  | von'])

X = pd.get_dummies(
    panel_exog,
    columns=['series_key'],
    drop_first=True
)

# 3) Sicherstellen, dass alles float ist:
X = X.astype('float64')

# 4) Endogen definieren
y = panel['consumption'].astype('float64')


In [9]:
print(panel.dtypes)

von  | von     datetime64[ns]
consumption           float64
hour                    int32
weekday                 int32
month                   int32
is_weekend              int64
w_tl                  float64
w_rf                  float64
w_ff                  float64
w_ffx                 float64
w_cglo                float64
w_so_h                float64
w_rr                  float64
w_rrm                 float64
w_tb10                float64
w_tb20                float64
CEGH_WAP              float64
THE_WAP               float64
s24                   float64
s168                  float64
s8760                 float64
series_key             object
dtype: object


In [10]:
print(X.dtypes.value_counts())

bool       50
float64    15
int32       3
int64       1
Name: count, dtype: int64


In [13]:
# Dictionary zum Speichern der Ergebnisse
results = {}

# Liste aller Exogenen Features
exog_vars = [
    'hour','weekday','month','is_weekend',
    'w_tl','w_rf','w_ff','w_ffx','w_cglo','w_so_h',
    'w_rr','w_rrm','w_tb10','w_tb20',
    'CEGH_WAP','THE_WAP',
    's24','s168','s8760'
]

for key, df in tqdm(series_dfs.items(), desc="Fitting pro Serie"):
    # 1) Index duplikatfrei & stündliche Frequenz erzwingen
    df = df[~df.index.duplicated(keep='first')]
    df = df.asfreq('h')

    # 2) MSTL-Seasonal-Features holen und joinen
    m = mstl_results[key].seasonal.rename(columns={
        'seasonal_24':   's24',
        'seasonal_168':  's168',
        'seasonal_8760': 's8760'
    })
    df_i = df.join(m, how='left')

    # 3) Komplettes Drop-NaN (inkl. MSTL-Kälte am Anfang)
    df_clean = df_i[['consumption'] + exog_vars].dropna()

    # 4) Index als lückenlose Stundenreihe neu aufbauen
    start   = df_clean.index.min()
    periods = len(df_clean)
    df_clean.index = pd.date_range(start=start, periods=periods, freq='h')

    # 5) SARIMAX definieren und fitten mit low_memory
    model_i = SARIMAX(
        endog = df_clean['consumption'].astype('float64'),
        exog  = df_clean[exog_vars].astype('float64'),
        order            = (1,0,1),
        seasonal_order   = (1,0,0,24),
        enforce_stationarity   = False,
        enforce_invertibility   = False
    )
    results[key] = model_i.fit(disp=False, low_memory=True)

Fitting pro Serie:   0%|          | 0/51 [00:00<?, ?it/s]C:\Users\johan\miniconda3\envs\ersatzwerte_project\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
Fitting pro Serie:   2%|▏         | 1/51 [20:09<16:48:18, 1209.97s/it]C:\Users\johan\miniconda3\envs\ersatzwerte_project\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
Fitting pro Serie:   4%|▍         | 2/51 [39:47<16:12:28, 1190.79s/it]C:\Users\johan\miniconda3\envs\ersatzwerte_project\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
Fitting pro Serie:   6%|▌         | 3/51 [59:46<15

In [15]:
results['FL_00024702.PLAN.MengeHH'].mle_retvals

{'fopt': np.float64(0.7955719494794584),
 'gopt': array([-0.0775598 , -0.02918506, -0.00292882, -0.01457219, -0.04659794,
         0.03590045,  0.04901243, -0.04816481,  0.19045068,  0.06267921,
         0.03877144, -0.00768081, -0.04405831,  0.00537182,  0.03907896,
         0.03728106, -0.10247029, -0.01411864, -0.02674244,  0.00386129,
        -0.06137141, -0.04163195,  0.04065647]),
 'fcalls': 1536,
 'warnflag': 1,
 'converged': False,
 'iterations': 50}

In [17]:
import pickle

with open('../data/sarimax_results.pkl', 'wb') as f:
    pickle.dump(results, f)

In [18]:
# Wähle genau eine Serie aus:
key = 'FL_00024702.PLAN.MengeHH'      # <-- hier deinen gewünschten Key eintragen
df = series_dfs[key]

# 1) Index duplikatfrei & stündlich ausrichten
df = df[~df.index.duplicated(keep='first')]
df = df.asfreq('h')

# 2) MSTL-Seasonal-Features holen und joinen
m = mstl_results[key].seasonal.rename(columns={
    'seasonal_24':   's24',
    'seasonal_168':  's168',
    'seasonal_8760': 's8760'
})
df_i = df.join(m, how='left')

# 3) Alle NaNs (inkl. MSTL-Edge) entfernen
exog_vars = [
    'hour','weekday','month','is_weekend',
    'w_tl','w_rf','w_ff','w_ffx','w_cglo','w_so_h',
    'w_rr','w_rrm','w_tb10','w_tb20',
    'CEGH_WAP','THE_WAP',
    's24','s168','s8760'
]
df_clean = df_i[['consumption'] + exog_vars].dropna()

# 4) Lückenlosen Stunden-Index neu aufbauen
start   = df_clean.index.min()
periods = len(df_clean)
df_clean.index = pd.date_range(start=start, periods=periods, freq='h')

In [19]:
model = SARIMAX(
    endog = df_clean['consumption'].astype('float64'),
    exog  = df_clean[exog_vars].astype('float64'),
    order            = (1,0,1),
    seasonal_order   = (1,0,0,24),
    enforce_stationarity   = False,
    enforce_invertibility   = False
)

In [20]:
res_single_nel = model.fit(
    disp=False,
    low_memory=True,
    method='nm',     # Nelder–Mead
    maxiter=200      # bis zu 200 Iterationen
)
print(res_single_nel.summary())

C:\Users\johan\miniconda3\envs\ersatzwerte_project\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                      SARIMAX Results                                      
Dep. Variable:                         consumption   No. Observations:                35114
Model:             SARIMAX(1, 0, 1)x(1, 0, [], 24)   Log Likelihood              -50547.823
Date:                             Mon, 21 Jul 2025   AIC                         101141.647
Time:                                     14:49:44   BIC                         101336.357
Sample:                                 01-01-2015   HQIC                        101203.659
                                      - 01-03-2019                                         
Covariance Type:                            approx                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
hour           0.0703      0.002     40.919      0.000       0.067       0.074
weekday        0.2495      

In [21]:
res_powell = model.fit(
    disp=False,
    low_memory=True,
    method='powell',
    maxiter=300
)
print(res_powell.summary())

                                      SARIMAX Results                                      
Dep. Variable:                         consumption   No. Observations:                35114
Model:             SARIMAX(1, 0, 1)x(1, 0, [], 24)   Log Likelihood              -24802.026
Date:                             Mon, 21 Jul 2025   AIC                          49650.052
Time:                                     15:07:06   BIC                          49844.762
Sample:                                 01-01-2015   HQIC                         49712.065
                                      - 01-03-2019                                         
Covariance Type:                            approx                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
hour           0.0004      0.000      0.891      0.373      -0.000       0.001
weekday       -0.0008      

In [22]:
# BFGS statt L-BFGS-B
res_bfgs = model.fit(
    disp=False,
    low_memory=True,
    method='bfgs',
    maxiter=200,
    tol=1e-5
)

print(res_bfgs.summary())

C:\Users\johan\miniconda3\envs\ersatzwerte_project\lib\site-packages\statsmodels\base\optimizer.py:19: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method bfgs is: gtol, norm, epsilon. The list of unsupported keyword arguments passed include: tol. After release 0.14, this will raise.
  warnings.warn(


                                      SARIMAX Results                                      
Dep. Variable:                         consumption   No. Observations:                35114
Model:             SARIMAX(1, 0, 1)x(1, 0, [], 24)   Log Likelihood              -24797.822
Date:                             Mon, 21 Jul 2025   AIC                          49641.645
Time:                                     15:27:05   BIC                          49836.354
Sample:                                 01-01-2015   HQIC                         49703.657
                                      - 01-03-2019                                         
Covariance Type:                            approx                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
hour           0.0004      0.000      0.892      0.372      -0.000       0.001
weekday       -0.0026      